In [2]:
import gurobipy as gp
from gurobipy import GRB

def solve_explanation_1_m(notes_winner, notes_loser, poids, matieres, epsilon=1e-4):
    """
    Résout le problème d'explication de type (1-m).
    Retourne le statut et l'explication si trouvée.
    """
    # 1. Calcul des contributions
    deltas = []
    for k in range(len(matieres)):
        val = poids[k] * (notes_winner[k] - notes_loser[k])
        deltas.append(val)

    # 2. Séparation Pros / Cons
    P = [i for i, d in enumerate(deltas) if d > 0]
    C = [j for j, d in enumerate(deltas) if d < 0]
    
    print(f"Pros (Indices): {P} -> {[matieres[i] for i in P]} (Val: {[deltas[i] for i in P]})")
    print(f"Cons (Indices): {C} -> {[matieres[j] for j in C]} (Val: {[deltas[j] for j in C]})")

    if not C:
        return "Trivial (Pas de Cons)", None
    
    # 3. Modèle Gurobi
    m = gp.Model("Explication_1_m")
    m.setParam('OutputFlag', 0) # Mode silencieux

    # Variables
    # x[i,j] = 1 si Pro i couvre Con j
    x = {}
    for i in P:
        for j in C:
            x[i, j] = m.addVar(vtype=GRB.BINARY, name=f"link_{i}_{j}")
    
    # y[i] = 1 si Pro i est utilisé
    y = {}
    for i in P:
        y[i] = m.addVar(vtype=GRB.BINARY, name=f"use_{i}")

    # Contrainte C1: Chaque Con doit être couvert exactement une fois
    for j in C:
        m.addConstr(gp.quicksum(x[i, j] for i in P) == 1, name=f"cover_{j}")

    # Contrainte C2: Lien x_ij <= y_i
    for i in P:
        for j in C:
            m.addConstr(x[i, j] <= y[i], name=f"link_logic_{i}_{j}")

    # Contrainte C3: Somme pondérée positive pour chaque groupe (1-m)
    # delta_i * y_i + sum(delta_j * x_ij) >= epsilon * y_i
    for i in P:
        sum_cons = gp.quicksum(deltas[j] * x[i, j] for j in C)
        m.addConstr(deltas[i] * y[i] + sum_cons >= epsilon * y[i], name=f"validity_{i}")

    # Résolution
    m.optimize()

    # Analyse
    if m.Status == GRB.OPTIMAL:
        explanation = {}
        for i in P:
            if y[i].X > 0.5:
                covered_cons = [matieres[j] for j in C if x[i, j].X > 0.5]
                explanation[matieres[i]] = covered_cons
        return "OPTIMAL", explanation
    elif m.Status == GRB.INFEASIBLE:
        return "INFEASIBLE", None
    else:
        return f"Status {m.Status}", None

# --- Données du problème (Tableau Page 2/3) ---
matieres = ["Anatomie", "Biologie", "Chirurgie", "Diagnostic", "Epidemiologie", "Forensic", "Génétique"]
poids = [8, 7, 7, 6, 6, 5, 6]

# Notes des candidats u et v (Source 40)
# u: 72, 66, 75, 85, 88, 66, 93
# v: 71, 73, 63, 92, 76, 79, 93
notes_w = [79, 69, 78, 76, 67, 84, 79]
notes_w_prime = [57, 76, 81, 76, 82, 86, 77]
notes_u = [72, 66, 75, 85, 88, 66, 93]
notes_v = [71, 73, 63, 92, 76, 79, 93]


print("--- Test de l'explication (1-m) pour w > w' ---")
status, result = solve_explanation_1_m(notes_w, notes_w_prime, poids, matieres)

print("-" * 30)
if status == "OPTIMAL":
    print("Succès ! Explication trouvée :")
    for pro, cons in result.items():
        print(f" - {pro} compense {cons}")
elif status == "INFEASIBLE":
    print("Résultat : INFEASIBLE")
    print("Preuve : Il n'existe pas d'explication de type (1-m) pour w > w'.")



print("--- Test de l'explication (1-m) pour u > v ---")
status, result = solve_explanation_1_m(notes_u, notes_v, poids, matieres)

print("-" * 30)
if status == "OPTIMAL":
    print("Succès ! Explication trouvée :")
    for pro, cons in result.items():
        print(f" - {pro} compense {cons}")
elif status == "INFEASIBLE":
    print("Résultat : INFEASIBLE")
    print("Preuve : Il n'existe pas d'explication de type (1-m) pour u > v.")

--- Test de l'explication (1-m) pour w > w' ---
Pros (Indices): [0, 6] -> ['Anatomie', 'Génétique'] (Val: [176, 12])
Cons (Indices): [1, 2, 4, 5] -> ['Biologie', 'Chirurgie', 'Epidemiologie', 'Forensic'] (Val: [-49, -21, -90, -10])
------------------------------
Succès ! Explication trouvée :
 - Anatomie compense ['Biologie', 'Chirurgie', 'Epidemiologie', 'Forensic']
 - Génétique compense []
--- Test de l'explication (1-m) pour u > v ---
Pros (Indices): [0, 2, 4] -> ['Anatomie', 'Chirurgie', 'Epidemiologie'] (Val: [8, 84, 72])
Cons (Indices): [1, 3, 5] -> ['Biologie', 'Diagnostic', 'Forensic'] (Val: [-49, -42, -65])
------------------------------
Résultat : INFEASIBLE
Preuve : Il n'existe pas d'explication de type (1-m) pour u > v.
